In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
import train
import argparse

parser = argparse.ArgumentParser()
parser.add_argument(
    '-n', '--name',
    type=str,
    default="missing name",
    help="Name of network",
    dest="name"
)
parser.add_argument(
    '-s', '--seed',
    type=float,
    default=1,
    help="Seed for randomness",
    dest="seed"
)
parser.add_argument(
    '-l', '--layer',
    type=str,
    default="all",
    help="layer_name, give all for all layers",
    dest="layer"
)

args=parser.parse_args()
model_str=args.name
layer_str=args.layer
tf.random.set_seed(args.seed)

exec(open(model_str+'/'+model_str+".py").read())
model.load_weights(model_str+'/saved_model/'+model_str)
model,weights=train.weight_measure(model,layer_str,10,final_learning_rate,x_train,y_train,batch_size)
np.save(model_str+'/weights',weights)